In [17]:
import sys 
sys.path.append("../utils")
from dump import dump
import numpy as np
import pandas as pd
import plotly
import plotly.express as px
import plotly.io as pio
pio.renderers.default = 'notebook'
import plotly.graph_objects as go
colors = px.colors.qualitative.G10

In [18]:
d = dump("../run/run.dump")
times = d.time()
dfs = []
for t in times:
    id, type, x, y, z, radius, d1, d2, d3 = d.vecs(t, "id", "type", "x", "y", "z", "radius", "d1", "d2", "d3")
    data = {"time":[t]*len(type),"id":id,"type":type,"x":x,"y":y,"z":z,"radius":radius,"d1":d1,"d2":d2,"d3":d3}
    dfs.append(pd.DataFrame(data))
df = pd.concat(dfs,ignore_index=True)
df["type"] = df["type"].astype(str)
df["typename"] = None
df.loc[df.type=="1.0", 'typename'] = "Sia"
df.loc[df.type=="2.0", 'typename'] = "Vac"
df.loc[df.type=="1.0", 'typecolor'] = colors[0]
df.loc[df.type=="2.0", 'typecolor'] = colors[1]
df["area"] = df.radius*df.radius
nsSia = []
nsVac = []
for time in times:
    nSia = len(df[(df.time==time) & (df.type=="1.0")].id)
    nVac = len(df[(df.time==time) & (df.type=="2.0")].id)
    nsSia.append(nSia)
    nsVac.append(nVac)

10000000 20000000 30000000 40000000 50000000 60000000 70000000 80000000 90000000 100000000 110000000 120000000 130000000 140000000 150000000 160000000 170000000 180000000 190000000 200000000 210000000 220000000 230000000 240000000 250000000 260000000 270000000 280000000 
read 28 snapshots
28 snapshots selected out of 28
assigned columns: id type x y z size d1 d2 d3 radius 
dump is already unscaled


In [16]:
df

,time,id,type,x,y,z,radius,typename,typecolor,area
0,10000000,2.0,2.0,146.235130,6.922556,172.312574,1.877753,Vac,#DC3912,3.525955
1,10000000,28.0,2.0,13.666667,54.000000,190.666667,1.877753,Vac,#DC3912,3.525955
2,10000000,30.0,2.0,199.333333,40.166667,57.333333,2.246905,Vac,#DC3912,5.048582
3,10000000,38.0,2.0,146.428571,73.285714,184.000000,2.341254,Vac,#DC3912,5.481470
4,10000000,46.0,2.0,181.956104,59.157922,145.592078,2.020690,Vac,#DC3912,4.083187
...,...,...,...,...,...,...,...,...,...,...
3246,220000000,21994.0,2.0,58.000000,30.000000,28.000000,1.442250,Vac,#DC3912,2.080084
3247,220000000,21995.0,1.0,39.000000,136.000000,18.000000,1.442250,Sia,#3366CC,2.080084
3248,220000000,21997.0,1.0,5.000000,187.000000,137.000000,1.442250,Sia,#3366CC,2.080084
3249,220000000,21998.0,2.0,165.000000,198.000000,181.000000,1.442250,Vac,#DC3912,2.080084


In [20]:
shown_tims = times[:]
frames = [
    go.Frame(
        data=[
            go.Scatter3d(
                x=df.x, y=df.y, z=df.z,  mode="markers", name = typename,
                marker = dict(
                    size= df.radius*2,
                    color = df.typecolor,
                    opacity = 1.0,
                    line = dict(
                        color='white',
                        width=6
                    )
                )
            )
            for df,typename in zip([df[df.typename==typename] for typename in ["Sia","Vac"]], ["SIA", "Vacancy"])
        ], 
        name=f"{time/1E9} B"
    )
    for df,time in zip([df[df.time==time] for time in shown_tims], shown_tims)
]

layout = go.Layout(
    sliders=[
        dict(
            steps = [
                dict(
                    args=[
                        [f.name], 
                        {
                            'frame': {
                                'duration': 0, 'redraw': True
                            }
                        }
                    ],
                label=f.name, 
                method='animate'
                ) 
                for f in frames
            ],
            currentvalue={"prefix": "Step: "}
        ),
    ],
    scene=dict(
        camera=dict(
            projection=dict(
                type="orthographic"
            )
        ),
        xaxis = dict(range=[-10,210],),
        yaxis = dict(range=[-10,210],),
        zaxis = dict(range=[-10,210],),
        aspectmode='cube',
        aspectratio=dict(x=1, y=1, z=1)
    ),
    width=800,
    height=600,
    legend= {'itemsizing': 'constant'}
)
    
fig = go.Figure(data=frames[0].data, frames=frames, layout=layout)
fig.show()

In [9]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=times, y=nsSia, name="SIA"))
fig.add_trace(go.Scatter(x=times, y=nsVac, name="Vacancy"))
fig.show()